In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation as LDA
import pyLDAvis
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pyLDAvis.sklearn

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
# 2016
reviews_2016 = pd.read_csv("reviews_2016.csv.gz")
listing_2016 = pd.read_csv("listing_2016.csv.gz")

# 2017
reviews_2017 = pd.read_csv("reviews_2017.csv.gz")

# 2018
reviews_2018 = pd.read_csv("reviews_2018.csv.gz")

# 2019
reviews_2019 = pd.read_csv("reviews_2019.csv.gz")

In [3]:
df = reviews_2016.append([reviews_2017, reviews_2018,reviews_2019])
df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3269390,14646413,2014-06-23,16547040,Anna,Diane is a wonderful host and was extremely ac...
1,3269390,14878133,2014-06-28,16503998,Lauren,"Loved our stay with Diane! Great location, wo..."
2,3269390,15060840,2014-07-01,512242,Kelly,This place was great it is exactly like the pi...
3,3269390,15412255,2014-07-08,10286257,Kimberly,Diane was a great host! Didn't get the chance ...
4,3269390,16717966,2014-08-01,12743532,Dawnyale,The listing was even better than the posting d...
...,...,...,...,...,...,...
419896,40568019,575370414,2019-12-09,164529575,Shirley,The host canceled this reservation 3 days befo...
419897,40616356,576894107,2019-12-14,314932885,Tyler,A beautiful little apartment! It shows that th...
419898,40806100,578731495,2019-12-18,112931800,Sylvia,Great last minute airbnb! Would absolutely rec...
419899,40839775,578441297,2019-12-17,98457970,Perry,Great place


In [4]:
df['year'] = df['date'].str[0:4]

In [5]:
df = df.astype({'year': 'int32'})

In [6]:
reviews = df[df.year == 2017]
reviews = reviews.loc[:, ['listing_id', 'date', 'year', 'comments']]
reviews = reviews.dropna()

reviews

from nltk.corpus import stopwords
stop = stopwords.words('english')

# Removing punctuation.
reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')

# Making text lower case.
reviews["comments"] = reviews["comments"].str.lower().str.split()

# Removing Stop-words.
reviews['comments'] = reviews['comments'].apply(lambda x: [item for item in x if item not in stop])


import nltk
from nltk.stem import WordNetLemmatizer 
# Init Lemmatizer
lemmatizer = WordNetLemmatizer()
reviews['comments'] = reviews['comments'].apply(lambda x: ([lemmatizer.lemmatize(w,) for w in x]))

reviews["comments"]= reviews["comments"].apply(lambda x: [item for item in x if item.isalpha()] )

reviews['comments'] = reviews['comments'].apply(lambda x: " ".join(x))

<>:11: DeprecationWarning: invalid escape sequence \w
<>:11: DeprecationWarning: invalid escape sequence \w
<>:11: DeprecationWarning: invalid escape sequence \w
<ipython-input-6-04dd922f3e15>:11: DeprecationWarning: invalid escape sequence \w
  reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')
/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(binary = True)

# define vectorizer
X = vectorizer.fit_transform(reviews.comments.values)

# vectorize your data. Note: this creates a sparce matrix, I used .toarray() to make it a dense matrix.
words = vectorizer.get_feature_names()
BagOfWords = pd.DataFrame.sparse.from_spmatrix(X)
BagOfWords.columns = words

words = vectorizer.get_feature_names()

In [8]:
frequency_words = BagOfWords.sum(axis = 0)

frequency_df = pd.DataFrame(frequency_words).sort_values(by=[0], ascending = False)

k = frequency_df[frequency_df[0]>10]
k.tail(30)

In [11]:
reviews = df[df.year == 2016]
len(reviews.listing_id.unique())

3139

In [12]:
reviews = reviews.loc[:, ['listing_id', 'date', 'year', 'comments']]
reviews = reviews.dropna()

from nltk.corpus import stopwords
stop = stopwords.words('english')

# Removing punctuation.
reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')

# Making text lower case.
reviews["comments"] = reviews["comments"].str.lower().str.split()

# Removing Stop-words.
reviews['comments'] = reviews['comments'].apply(lambda x: [item for item in x if item not in stop])

import nltk
from nltk.stem import WordNetLemmatizer 

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()
reviews['comments'] = reviews['comments'].apply(lambda x: ([lemmatizer.lemmatize(w,) for w in x]))

reviews["comments"]= reviews["comments"].apply(lambda x: [item for item in x if item.isalpha()] )

reviews['comments'] = reviews['comments'].apply(lambda x: " ".join(x))

##########################################

k = pd.DataFrame(reviews.comments[:5000])
vectorizer = CountVectorizer(binary = True)
# define vectorizer
X = vectorizer.fit(k.comments.values)
counts = vectorizer.transform(k.comments)
counts
lda = LDA(n_components = 3, random_state = 11)
lda.fit(counts)
ldaOut = lda.transform(counts)
p = pyLDAvis.sklearn.prepare(lda,counts,vectorizer)
pyLDAvis.save_html(p,"Reviews.html")

<>:8: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \w
<ipython-input-12-3de40a2afc6d>:8: DeprecationWarning: invalid escape sequence \w
  reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')
/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


### ADDING DROPOUT 2017

In [13]:
listings2017=pd.read_csv("listing_2016.csv.gz")
listings2018=pd.read_csv("listing_2018.csv.gz")
listings2019=pd.read_csv("listing_2019.csv.gz")

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
set_19 = set(listings2019.id)
set_18 = set(listings2018.id)
set_17 = set(listings2017.id)

listings2018['drop_out'] = listings2018['id'].apply(lambda x: 0 if x in set_19 else 1)
listings2017['drop_out'] = listings2017['id'].apply(lambda x: 0 if x in set_18 else 1)

In [15]:
listings2017 = listings2017.loc[:,['id', 'drop_out']]
A = set(listings2017.id)

In [16]:
reviews = df[df.year == 2016]
reviews['id'] = reviews['listing_id']
reviews = reviews.loc[:, ['id', 'date', 'year', 'comments']]
reviews = reviews.dropna()
B = set(reviews.id)

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
reviews = reviews.merge(listings2017, left_on='id', right_on='id')
reviews

,id,date,year,comments,drop_out
0,7202016,2016-01-02,2016,Kelly was an amazing host who made the stay ve...,1
1,6939472,2016-01-03,2016,Very good value. Near public train station. Cl...,0
2,6939472,2016-01-03,2016,Very good value. Near public train station. Cl...,0
3,6939472,2016-01-09,2016,They were too bad host.\nI've never met this k...,0
4,6939472,2016-01-18,2016,"Nice, clean, friendly and extremely convenient...",0
...,...,...,...,...,...
73000,10068879,2016-07-30,2016,The host canceled this reservation 49 days bef...,1
73001,10068879,2016-08-06,2016,"Jim and Mary Ann are really great hosts, \r\nT...",1
73002,10068879,2016-08-14,2016,"Very enjoyable place, nice environment and fri...",1
73003,10319529,2016-02-15,2016,Lovely place and warm welcome. Nice neighborho...,1


In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Removing punctuation.
reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')

# Making text lower case.
reviews["comments"] = reviews["comments"].str.lower().str.split()

# Removing Stop-words.
reviews['comments'] = reviews['comments'].apply(lambda x: [item for item in x if item not in stop])

import nltk
from nltk.stem import WordNetLemmatizer 

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()
reviews['comments'] = reviews['comments'].apply(lambda x: ([lemmatizer.lemmatize(w,) for w in x]))

reviews["comments"]= reviews["comments"].apply(lambda x: [item for item in x if item.isalpha()] )

reviews['comments'] = reviews['comments'].apply(lambda x: " ".join(x))

<>:5: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \w
<ipython-input-18-6f7b8d172882>:5: DeprecationWarning: invalid escape sequence \w
  reviews['comments'] = reviews['comments'].str.replace('[^\w\s]','')


In [19]:
zeros = reviews[reviews.drop_out == 0]
ones = reviews[reviews.drop_out == 1]

In [23]:
# LDA on Zeros
k = pd.DataFrame(zeros.comments)
vectorizer = CountVectorizer(binary = True)
# define vectorizer
X = vectorizer.fit(k.comments.values)
counts = vectorizer.transform(k.comments)
lda = LDA(n_components = 4, random_state = 11)
lda.fit(counts)
ldaOut = lda.transform(counts)
p = pyLDAvis.sklearn.prepare(lda,counts,vectorizer)
pyLDAvis.save_html(p,"Reviews_zeros.html")

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [21]:
# LDA on ones
k = pd.DataFrame(ones.comments)
vectorizer = CountVectorizer(binary = True)
# define vectorizer
X = vectorizer.fit(k.comments.values)
counts = vectorizer.transform(k.comments)
lda = LDA(n_components = 4, random_state = 11)
lda.fit(counts)
ldaOut = lda.transform(counts)
p = pyLDAvis.sklearn.prepare(lda,counts,vectorizer)
pyLDAvis.save_html(p,"Reviews_ones.html")

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [21]:
r = df[df.year == 2018]
r['id'] = r['listing_id']
r = r.loc[:, ['id', 'date', 'year', 'comments']]
r = r.dropna()
B = set(r.id)

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
r = r.merge(listings2018, left_on='id', right_on='id')
r

,id,date,year,comments,listing_url,scrape_id,last_scraped,name,summary,space,...,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,drop_out
0,2318,2018-02-20,2018,"Lovely, sunny house. Fully equipped kitchen. ...",https://www.airbnb.com/rooms/2318,20181213042428,2018-12-13,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...",...,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,3,0.17,0
1,2318,2018-02-25,2018,This house made me want to move to Madrona :)....,https://www.airbnb.com/rooms/2318,20181213042428,2018-12-13,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...",...,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,3,0.17,0
2,2318,2018-03-28,2018,Casa Madrona was perfect for our family of 5! ...,https://www.airbnb.com/rooms/2318,20181213042428,2018-12-13,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...",...,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,3,0.17,0
3,2318,2018-04-04,2018,"Megan's house is spacious, attractive, ultra-c...",https://www.airbnb.com/rooms/2318,20181213042428,2018-12-13,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...",...,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,3,0.17,0
4,2318,2018-04-22,2018,"While in Seattle for a couple weeks, I rented ...",https://www.airbnb.com/rooms/2318,20181213042428,2018-12-13,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...",...,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,3,0.17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269579,30750171,2018-12-23,2018,"Amazing apartment, my fiancé absolutely loved ...",https://www.airbnb.com/rooms/30750171,20181213042428,2018-12-13,Modern 1908 Charm in the Heart of Capitol Hill,Right in the heart of Seattle's Capitol Hill n...,This home-y apartment is located on a tree lin...,...,NaN,{WASHINGTON},t,f,moderate,f,f,1,NaN,0
269580,30750171,2018-12-28,2018,Really cute little spot to hang your hat when ...,https://www.airbnb.com/rooms/30750171,20181213042428,2018-12-13,Modern 1908 Charm in the Heart of Capitol Hill,Right in the heart of Seattle's Capitol Hill n...,This home-y apartment is located on a tree lin...,...,NaN,{WASHINGTON},t,f,moderate,f,f,1,NaN,0
269581,30772471,2018-12-25,2018,"Beautiful home, very cozy and clean. A+ for cl...",https://www.airbnb.com/rooms/30772471,20181213042428,2018-12-13,Charming Seattle retreat in Beacon Hill,Beautifully furnished 2680 sqft gem nestled in...,COMMON AREAS: - Spacious open living room equi...,...,NaN,{WASHINGTON},t,f,moderate,f,f,1,NaN,0
269582,30772471,2018-12-26,2018,"Home was beautiful, spacious and very clean fo...",https://www.airbnb.com/rooms/30772471,20181213042428,2018-12-13,Charming Seattle retreat in Beacon Hill,Beautifully furnished 2680 sqft gem nestled in...,COMMON AREAS: - Spacious open living room equi...,...,NaN,{WASHINGTON},t,f,moderate,f,f,1,NaN,0


In [23]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Removing punctuation.
r['comments'] = r['comments'].str.replace('[^\w\s]','')

# Making text lower case.
r["comments"] = r["comments"].str.lower().str.split()

# r Stop-words.
reviews['comments'] = r['comments'].apply(lambda x: [item for item in x if item not in stop])

import nltk
from nltk.stem import WordNetLemmatizer 

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()
r['comments'] = r['comments'].apply(lambda x: ([lemmatizer.lemmatize(w,) for w in x]))

r["comments"]= r["comments"].apply(lambda x: [item for item in x if item.isalpha()] )

r['comments'] = r['comments'].apply(lambda x: " ".join(x))

<>:5: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \w
<ipython-input-23-5b3ece1942b4>:5: DeprecationWarning: invalid escape sequence \w
  r['comments'] = r['comments'].str.replace('[^\w\s]','')


In [24]:
newzeros = r[r.drop_out == 0]
newones = r[r.drop_out == 1]

newzeros.to_csv("2018_notdropped.csv")
newones.to_csv("2018_dropped.csv")


## Ameneties

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords
stop = stopwords.words('english')
listings2017=pd.read_csv("listing_2016.csv.gz")
listings2018=pd.read_csv("listing_2018.csv.gz")
listings2019=pd.read_csv("listing_2019.csv.gz")

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
new = pd.DataFrame()
new["amenities"] = listings2017["amenities"].str.lower()
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(new.amenities.values)
words = vectorizer.get_feature_names()
BagOfWords = pd.DataFrame.sparse.from_spmatrix(X)
BagOfWords.columns = words
words_2017= vectorizer.get_feature_names()
words = [x for x in words if x not in stop]
BagOfWords = BagOfWords.loc[:,words]
frequency_words = BagOfWords.sum(axis = 0)
frequency_df = pd.DataFrame(frequency_words).sort_values(by=[0], ascending = False)
k = frequency_df[frequency_df[0]>0]
BagOfWords = BagOfWords.loc[:,words]
BagOfWords

,24,accessible,aid,air,allowed,bedroom,breakfast,building,buzzer,cable,...,shampoo,smoke,smoking,suitable,tub,tv,washer,wheelchair,wireless,workspace
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,1,0
1,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,1,0
2,0,0,0,1,1,0,0,0,0,1,...,1,1,0,0,1,1,1,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,1,0
4,0,0,1,0,0,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,1,0,1,0,0,0,0,0,1,...,1,1,0,0,0,1,1,1,1,0
3814,1,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,1
3815,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,1,1,0,1,1
3816,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,1


,24,accessible,aid,air,allowed,bedroom,breakfast,building,buzzer,cable,...,shampoo,smoke,smoking,suitable,tub,tv,washer,wheelchair,wireless,workspace
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,1,0
1,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,1,0
2,0,0,0,1,1,0,0,0,0,1,...,1,1,0,0,1,1,1,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,1,0
4,0,0,1,0,0,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,1,0,1,0,0,0,0,0,1,...,1,1,0,0,0,1,1,1,1,0
3814,1,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,1,0,0,1,1
3815,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,1,1,0,1,1
3816,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,1


In [34]:
new = pd.DataFrame()
new["amenities"] = listings2018["amenities"].str.lower()
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(new.amenities.values)
words = vectorizer.get_feature_names()
BagOfWords = pd.DataFrame.sparse.from_spmatrix(X)
BagOfWords.columns = words
words = vectorizer.get_feature_names()
words = [x for x in words if x not in stop]
BagOfWords.loc[:,words]

,24,access,accessible,aid,air,alfresco,allowed,alone,amazon,area,...,water,waterfront,well,wheelchair,wide,wifi,window,wine,wireless,workspace
0,0,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8590,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
8591,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
8592,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [43]:
new = pd.DataFrame()
new["amenities"] = listings2019["amenities"].str.lower()
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(new.amenities.values)
words = vectorizer.get_feature_names()
BagOfWords = pd.DataFrame.sparse.from_spmatrix(X)
BagOfWords.columns = words
words = vectorizer.get_feature_names()
words = [x for x in words if x not in stop]
BagOfWords.loc[:,words]

,24,access,accessible,aid,air,alfresco,allowed,alone,amazon,area,...,water,waterfront,well,wheelchair,wide,wifi,window,wine,wireless,workspace
0,0,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
1,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,0,0,0,1,1,0,1,0,1,0,...,1,0,1,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7964,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
7965,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
7966,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [50]:
frequency_words = BagOfWords.sum(axis = 0)
frequency_df = pd.DataFrame(frequency_words).sort_values(by=[0], ascending = False)
k = frequency_df[frequency_df[0]>50]
k.tail(50)

,0
gas,60
memory,60
foam,60
window,57
monitor,56
step,56
baking,53
sheet,53
suite,52
beachfront,50


In [6]:
listings2017=pd.read_csv("listing_2016.csv.gz")
listings2018=pd.read_csv("listing_2018.csv.gz")
listings2019=pd.read_csv("listing_2019.csv.gz")

/Users/aniruddh/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
new = pd.DataFrame()
new["amenities"] = listings2017["amenities"].str.lower()
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(new.amenities.values)
words = vectorizer.get_feature_names()
BagOfWords = pd.DataFrame.sparse.from_spmatrix(X)
BagOfWords.columns = words
words = [x for x in words if x not in stop]
BagOfWords = BagOfWords.loc[:,words]
frequency_words = BagOfWords.sum(axis = 0)
frequency_df = pd.DataFrame(frequency_words).sort_values(by=[0], ascending = False)
k = frequency_df[frequency_df[0]>0]
BagOfWords_17 = BagOfWords.loc[:,words]
BagOfWords_17['index'] = listings2017.id
BagOfWords_17

,24,accessible,aid,air,allowed,bedroom,breakfast,building,buzzer,cable,...,smoke,smoking,suitable,tub,tv,washer,wheelchair,wireless,workspace,index
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,1,0,1,0,241032
1,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1,1,0,1,0,953595
2,0,0,0,1,1,0,0,0,0,1,...,1,0,0,1,1,1,0,1,0,3308979
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,7421966
4,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,1,0,278830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,1,0,1,0,0,0,0,0,1,...,1,0,0,0,1,1,1,1,0,8101950
3814,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,1,1,8902327
3815,0,0,1,0,0,0,1,0,0,1,...,1,0,0,0,1,1,0,1,1,10267360
3816,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,1,1,9604740


In [3]:
listings2017.id

0         241032
1         953595
2        3308979
3        7421966
4         278830
          ...   
3813     8101950
3814     8902327
3815    10267360
3816     9604740
3817    10208623
Name: id, Length: 3818, dtype: int64